<a href="https://colab.research.google.com/github/murphycollins/murphycollins.github.io/blob/main/sales-forecasting/notebooks/sales_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Install required packages
!pip install pandas matplotlib prophet statsmodels seaborn

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# Load dataset (make sure sales_sample.csv exists in your repo under data/)
url = "https://raw.githubusercontent.com/murphycollins/sales-forecasting/main/data/sales_sample.csv"
df = pd.read_csv(url, parse_dates=['date'])

df.head()

In [ ]:

# Aggregate to monthly sales
df = df.sort_values("date")
df.set_index("date", inplace=True)
monthly = df['sales'].resample('MS').sum().reset_index()
monthly.columns = ["ds", "y"]

# Plot historical sales
plt.figure(figsize=(10,5))
plt.plot(monthly['ds'], monthly['y'], marker='o')
plt.title("Monthly Sales")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.show()

In [ ]:

try:
    from prophet import Prophet

    m = Prophet()
    m.fit(monthly)

    future = m.make_future_dataframe(periods=6, freq="MS")
    forecast = m.predict(future)

    # Plot forecast
    fig = m.plot(forecast)
    plt.show()

    forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(10)

except Exception as e:
    print("Prophet failed, using ARIMA instead:", e)
    from statsmodels.tsa.arima.model import ARIMA

    model = ARIMA(monthly['y'], order=(1,1,1))
    res = model.fit()
    forecast = res.forecast(steps=6)
    print("ARIMA forecast:", forecast)